# Data preprocessing
Erik Matovič and Jakub Horvat

[Dataset](https://data.nvision2.eecs.yorku.ca/PIE_dataset/)

### 0. Imports

In [91]:
import pandas as pd
from matplotlib import pyplot as plt
from os import listdir
from os.path import join
from pie_data import PIE
import cv2
from math import floor, ceil

In [92]:
pie_path = '../../zadanie2/' #'/media/erik/T7/DP/PIE_data'
pie = PIE(data_path=pie_path)
type(pie)

pie_data.PIE

In [93]:
train_list = pie._get_image_set_ids('train')
test_list = pie._get_image_set_ids('test')
val_list = pie._get_image_set_ids('val')

train_list, val_list, test_list

(['set01', 'set02', 'set04'], ['set05', 'set06'], ['set03'])

In [94]:
class PIE_peds():
    def __init__(self, setname: str, pie: PIE) -> None:
        #super().__init__()
        self.setname = setname
        self.set_dir = dict()
        self.all_path = list()
        self.all_filenames = list()
        self.path = pie._images_path

        assert setname in ['train', 'val', 'test']
        
        self.img_init(pie)
        
    
    def __len__(self):
        return len(self.all_filenames)
    
    def img_init(self, pie) -> None:
        set_list = pie._get_image_set_ids(self.setname)

        for set_ in set_list:
            set_path = join(pie._images_path, set_)
            self.set_dir[set_] = dict()
            
            set_content = listdir(set_path)
            for video in set_content:
                video_path = join(set_path, video)
                imgs = listdir(video_path)
                self.all_path += ([video_path] * len(imgs))
                self.all_filenames += imgs
                self.set_dir[set_][video] = imgs
        print(self.set_dir)    


In [95]:
train_dataset = PIE_peds('train', pie)
val_dataset = PIE_peds('val', pie)
test_dataset = PIE_peds('test', pie)

print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))

{'set01': {'video_0004': ['00002.png', '00005.png', '00001.png', '00000.png', '00013.png', '00003.png', '00006.png', '00007.png', '00008.png', '00015.png', '00010.png', '00004.png', '00011.png', '00014.png', '00009.png', '00012.png'], 'video_0001': ['01127.png', '01130.png', '01134.png', '01129.png', '01139.png', '01137.png', '01132.png', '01128.png', '01131.png', '01133.png', '01140.png', '01138.png', '01125.png', '01135.png', '01136.png', '01126.png'], 'video_0003': ['00127.png', '00128.png', '00121.png', '00134.png', '00131.png', '00123.png', '00135.png', '00129.png', '00130.png', '00132.png', '00124.png', '00126.png', '00122.png', '00125.png', '00133.png', '00120.png'], 'video_0002': ['04698.png', '04697.png', '04702.png', '04704.png', '04709.png', '04707.png', '04705.png', '04710.png', '04695.png', '04699.png', '04696.png', '04703.png', '04701.png', '04706.png', '04700.png', '04708.png']}, 'set02': {'video_0001': ['02830.png', '02836.png', '02838.png', '02825.png', '02823.png', '0

In [96]:
def convert(size, box):
    dw = 1./(size[0])
    dh = 1./(size[1])

    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw #- 0.001
    y = y*dh
    h = h*dh #- 0.001

    # floating rounding of BBoxes
    if (x + w/2) > 1:
        while (x + w/2) > 1.0001:
            x -= 0.0001
        
    if (x - w/2) < 0:
        while (x - w/2) < 0.0001:
            x += 0.0001
        #print(x)
        #x = 0.001
        #print(x)

    if (y + h/2) > 1:
        while (y + h/2) > 1.0001:
            y -= 0.0001
        
    if (y - h/2) < 0:
        while (y - h/2) < 0.0001:
            y += 0.0001

    #if float(w)-0.001 > 0:
    #    w = float(w)-0.001 #w
    #if float(h)-0.001 > 0:
    #    h = float(h)-0.001 #h

    return (x, y, w, h)

In [97]:
def save_img_annotations(dataset: PIE_peds, folder: str):# -> None:
    """
    Save images into given folder and also CSV annotations. 

    :param dataset: PIE_peds class, where all paths are set. 
    :param folder: folder train or val or test to save pedestrians.
    """

    assert folder in ['train', 'val', 'test']

    img_width = 1920
    img_height = 1080

    target = dict()
    target['set'] = []
    target['video'] = []
    target['frame'] = []
    target['Category'] = []
    target['x1'] = []
    target['y1'] = []
    target['width'] = []
    target['height'] = []

    target_v = dict()
    target_v['set'] = []
    target_v['video'] = []
    target_v['frame'] = []
    target_v['Category'] = []
    target_v['x1'] = []
    target_v['y1'] = []
    target_v['width'] = []
    target_v['height'] = []

    target_l = dict()
    target_l['set'] = []
    target_l['video'] = []
    target_l['frame'] = []
    target_l['Category'] = []
    target_l['x1'] = []
    target_l['y1'] = []
    target_l['width'] = []
    target_l['height'] = []

    target_s = dict()
    target_s['set'] = []
    target_s['video'] = []
    target_s['frame'] = []
    target_s['Category'] = []
    target_s['x1'] = []
    target_s['y1'] = []
    target_s['width'] = []
    target_s['height'] = []
    
    category = dict()
    category['p'] = 0   # pedestrian
    category['v'] = 1   # vehicle
    category['l'] = 2   # traffic light
    category['s'] = 3   # sign
    #img_counter = 0
    for set_name in dataset.set_dir:
        for video in dataset.set_dir[set_name]:
            annotations = pie._get_annotations(set_name, video)
            #annotations_attributes = pie._get_ped_attributes(set_name, video)
            for frame in dataset.set_dir[set_name][video]: 
                #img_path = '../images/' + set_name + '/' + video + '/' + frame
                #img_path = '/media/erik/T7/DP/PIE_data/images/' + set_name + '/' + video + '/' + frame
                #print(img_path, type(img_path))

                #img = cv2.imread(img_path)
                # plt.imshow(img)
                for idx in annotations['ped_annotations']: #['ped_annotations']:
                    frame_idx = int(frame[:-4])
                    #print(idx)
                    if frame_idx in annotations['ped_annotations'][idx]['frames']:
                        #if idx in target['ped']:
                        #    break

                        frame_key = annotations['ped_annotations'][idx]['frames'].index(frame_idx)
                        
                        bbox = annotations['ped_annotations'][idx]['bbox'][frame_key] 
                        
                        # BBox for pedestrian
                        x1 = bbox[0]
                        y1 = bbox[1]
                        x2 = bbox[2]
                        y2 = bbox[3]
                        width = x2 - x1
                        height = y2 - y1

                        x, y, w, h = convert((img_width, img_height), (x1, x2, y1, y2))
    
                        target['set'].append(set_name)
                        target['video'].append(video)
                        target['frame'].append(frame[:-4])
                        target['Category'].append(category['p'])
                        target['x1'].append(x)
                        target['y1'].append(y)
                        target['width'].append(w)
                        target['height'].append(h)
                        #print(idx + 'tl')
                        #print(annotations['traffic_annotations'])

                        #print(target)
                        #print(width / img_width, height / img_height)
                        #print(x1 / img_width, y1 / img_height)
                        #return 
                        #crop_img = img[y1:y2, x1:x2]
                        #crop_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
                        #crop_img = cv2.resize(crop_img, (64, 64))
                        
                        #f_name = '../images/' + folder + '/' + set_name + '_' + video + '_' + target['ped'][-1] + '_' + frame
                        #f_name = '/media/erik/T7/DP/PIE_data/images/' + folder + '/' + set_name + '_' + video + '_' + target['ped'][-1] + '_' + frame
                        
                        # print(f_name)
                        #cv2.imwrite(f_name, img)

                        #cv2.imshow("cropped", crop_img)
                        #cv2.waitKey(0)
                        #cv2.destroyAllWindows()
                        #print(type(target['BBox'][-1][1]))
                        #break
                for idx in annotations['traffic_annotations']:
                    frame_idx = int(frame[:-4])
                    #print(frame[:-4]) #idx)
                    #return
                    if frame_idx in annotations['traffic_annotations'][idx]['frames']: #['ped_annotations'][idx]['frames']:
                        #if idx in target['ped']:
                        #    break

                        frame_key = annotations['traffic_annotations'][idx]['frames'].index(frame_idx)#['ped_annotations'][idx]['frames'].index(frame_idx)
                        
                        bbox = annotations['traffic_annotations'][idx]['bbox'][frame_key] 
                        #bbox = annotations['ped_annotations'][idx]['bbox'][frame_key] 
                        #print(bbox)
                        # BBox for pedestrian
                        x1 = bbox[0]
                        y1 = bbox[1]
                        x2 = bbox[2]
                        y2 = bbox[3]

                        x, y, w, h = convert((img_width, img_height), (x1, x2, y1, y2))
                        #break
                        if idx[-1:] == 'v':
                            target_v['set'].append(set_name)
                            target_v['video'].append(video)
                            target_v['frame'].append(frame[:-4])
                            target_v['Category'].append(category['v'])
                            target_v['x1'].append(x)
                            target_v['y1'].append(y)
                            target_v['width'].append(w)
                            target_v['height'].append(h)
                        
                        elif idx[-1:] == 'l':
                            target_l['set'].append(set_name)
                            target_l['video'].append(video)
                            target_l['frame'].append(frame[:-4])
                            target_l['Category'].append(category['l'])
                            target_l['x1'].append(x)
                            target_l['y1'].append(y)
                            target_l['width'].append(w)
                            target_l['height'].append(h)

                        elif idx[-1:] == 's':
                            target_s['set'].append(set_name)
                            target_s['video'].append(video)
                            target_s['frame'].append(frame[:-4])
                            target_s['Category'].append(category['s'])
                            target_s['x1'].append(x)
                            target_s['y1'].append(y)
                            target_s['width'].append(w)
                            target_s['height'].append(h)
    #df = pd.DataFrame(data=target)
    #annotations_path = '../' + folder + '_annotations.csv'
    #df.to_csv(annotations_path)
    #print(f'{folder} done!')
    #print('target', target)
    #print('target_v', target_v)
    #print('target_l', target_l)
    #print('target_s', target_s)
    return target, target_v, target_l, target_s

In [98]:
target, target_v, target_l, target_s = save_img_annotations(train_dataset, 'train')

../../zadanie2/annotations/set01/video_0004_annt.xml
../../zadanie2/annotations/set01/video_0001_annt.xml
../../zadanie2/annotations/set01/video_0003_annt.xml
../../zadanie2/annotations/set01/video_0002_annt.xml
../../zadanie2/annotations/set02/video_0001_annt.xml
../../zadanie2/annotations/set02/video_0003_annt.xml
../../zadanie2/annotations/set02/video_0002_annt.xml
../../zadanie2/annotations/set04/video_0005_annt.xml
../../zadanie2/annotations/set04/video_0012_annt.xml
../../zadanie2/annotations/set04/video_0011_annt.xml
../../zadanie2/annotations/set04/video_0009_annt.xml
../../zadanie2/annotations/set04/video_0015_annt.xml
../../zadanie2/annotations/set04/video_0016_annt.xml
../../zadanie2/annotations/set04/video_0014_annt.xml
../../zadanie2/annotations/set04/video_0004_annt.xml
../../zadanie2/annotations/set04/video_0001_annt.xml
../../zadanie2/annotations/set04/video_0008_annt.xml
../../zadanie2/annotations/set04/video_0006_annt.xml
../../zadanie2/annotations/set04/video_0003_an

In [99]:
target = pd.DataFrame(target)
target_v = pd.DataFrame(target_v)
target_l = pd.DataFrame(target_l)
target_s = pd.DataFrame(target_s)

In [100]:
result = pd.concat([target, target_v, target_l, target_s])
result

,set,video,frame,Category,x1,y1,width,height
0,set01,video_0004,00002,0,0.561896,0.754509,0.004729,0.024000
1,set01,video_0004,00005,0,0.563677,0.755046,0.005031,0.024648
2,set01,video_0004,00001,0,0.561302,0.754329,0.004625,0.023769
3,set01,video_0004,00000,0,0.560706,0.754148,0.004526,0.023556
4,set01,video_0004,00013,0,0.567464,0.754755,0.004990,0.026231
...,...,...,...,...,...,...,...,...
75,set04,video_0013,01743,3,0.598328,0.612426,0.009219,0.084370
76,set04,video_0013,01748,3,0.750406,0.653889,0.169896,0.421019
77,set04,video_0013,01748,3,0.610648,0.611343,0.009745,0.087389
78,set04,video_0013,01737,3,0.712112,0.653074,0.153672,0.381111


In [101]:
#result.to_csv('../annotations.csv') #, index=False, header=False)

In [102]:
def maka_dataset(result:pd.DataFrame, folder:str):
    #folder = 'train'

    map_ = dict()
    map_['img'] = []
    map_['annotation'] = []

    for index, row in result.iterrows():
        set_ = row['set']           
        video = row['video']
        frame = row['frame']
        #img_path = '../../zadanie2/images/' + str(set_) + '/' + video + '/' + frame + '.png'
        #img_path = '/media/erik/T7/DP/PIE_data/images/' + str(set_) + '/' + video + '/' + frame + '.png'
        #print(img_path)
        #img = cv2.imread(img_path)
        #img_resize = cv2.resize(img, (600, 400))
        #cv2.imshow("img", img_resize)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        #print(set, video, frame, '\n', row)
        #f_name = '/media/erik/T7/DP/PIE_data/data/' + folder + '/' + str(index) + '.png'
        #print(f_name)
        #f_name = '/media/erik/T7/DP/PIE_data/images/' + folder + '/' + set_name + '_' + video + '_' + target['ped'][-1] + '_' + frame
        #print(set_ + '/' + video + '/' + frame + '.png')
        #break
        #cv2.imwrite(f_name, img)
        map_['img'].append(set_ + '/' + video + '/' + frame + '.png')
        
        df = result.loc[(result['set'] == set_) & 
                        (result['video'] == video) &
                        (result['frame'] == frame)].iloc[:, -5:]
        
        #annotations_path = '/media/erik/T7/DP/PIE_data/label/' + set_ + '/' + video + '/' + frame + '.csv'
        annotations_path = '../label/' + set_ + '/' + video + '/' + frame + '.txt'

        #specify path for export
        #path = r'c:\data_folder\my_data.txt'

        #export DataFrame to text file
        with open(annotations_path, 'w') as f:
            df_string = df.to_string(header=False, index=False)
            f.write(df_string)
        #df.to_csv(annotations_path, index=False, header=False)
        #print(f_name, annotations_path)

        map_['annotation'].append(set_ + '/' + video + '/' + frame + '.txt')
        #print(df)
        #break

    #path = '/media/erik/T7/DP/PIE_data/' + folder + '.csv'
    path = '../' + folder + '.csv'
    df = pd.DataFrame(map_)
    df.to_csv(path, index=False, header=False)

In [103]:
maka_dataset(result, 'train')

In [104]:
target, target_v, target_l, target_s = save_img_annotations(val_dataset, 'val')
target = pd.DataFrame(target)
target_v = pd.DataFrame(target_v)
target_l = pd.DataFrame(target_l)
target_s = pd.DataFrame(target_s)
result = pd.concat([target, target_v, target_l, target_s])
maka_dataset(result, 'val')

../../zadanie2/annotations/set05/video_0001_annt.xml
../../zadanie2/annotations/set05/video_0002_annt.xml
../../zadanie2/annotations/set06/video_0005_annt.xml
../../zadanie2/annotations/set06/video_0009_annt.xml
../../zadanie2/annotations/set06/video_0004_annt.xml
../../zadanie2/annotations/set06/video_0001_annt.xml
../../zadanie2/annotations/set06/video_0008_annt.xml
../../zadanie2/annotations/set06/video_0006_annt.xml
../../zadanie2/annotations/set06/video_0003_annt.xml
../../zadanie2/annotations/set06/video_0007_annt.xml
../../zadanie2/annotations/set06/video_0002_annt.xml


In [105]:
target, target_v, target_l, target_s = save_img_annotations(test_dataset, 'test')
target = pd.DataFrame(target)
target_v = pd.DataFrame(target_v)
target_l = pd.DataFrame(target_l)
target_s = pd.DataFrame(target_s)
result = pd.concat([target, target_v, target_l, target_s])
maka_dataset(result, 'test')

../../zadanie2/annotations/set03/video_0005_annt.xml
../../zadanie2/annotations/set03/video_0012_annt.xml
../../zadanie2/annotations/set03/video_0011_annt.xml
../../zadanie2/annotations/set03/video_0009_annt.xml
../../zadanie2/annotations/set03/video_0015_annt.xml
../../zadanie2/annotations/set03/video_0016_annt.xml
../../zadanie2/annotations/set03/video_0014_annt.xml
../../zadanie2/annotations/set03/video_0004_annt.xml
../../zadanie2/annotations/set03/video_0001_annt.xml
../../zadanie2/annotations/set03/video_0008_annt.xml
../../zadanie2/annotations/set03/video_0006_annt.xml
../../zadanie2/annotations/set03/video_0003_annt.xml
../../zadanie2/annotations/set03/video_0010_annt.xml
../../zadanie2/annotations/set03/video_0007_annt.xml
../../zadanie2/annotations/set03/video_0002_annt.xml
../../zadanie2/annotations/set03/video_0017_annt.xml
../../zadanie2/annotations/set03/video_0019_annt.xml
../../zadanie2/annotations/set03/video_0018_annt.xml
../../zadanie2/annotations/set03/video_0013_an